In [8]:
import torch
from models.regression_model import RModel
import pandas as pd
import random
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [12]:
df = pd.read_csv('data/num_train_processed.csv')
train_df = df[:1168];
test_df = df[1168:]

In [14]:
train_df.iloc[:,-1]

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1163    108959
1164    194000
1165    233170
1166    245350
1167    173000
Name: Saleprice, Length: 1168, dtype: int64

In [13]:
X = torch.tensor(train_df.iloc[:,1:-2].values,device=device)
Y = torch.tensor(train_df.iloc[:,-1].values,device=device)
test_x = torch.tensor(test_df.iloc[:,1:-2].values,device=device)
test_y = torch.tensor(test_df.iloc[:,-1].values,device=device)
rmodel = RModel(X.shape[1])
rmodel.to(device)

RModel(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=35, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=30, bias=True)
    (3): ReLU()
    (4): Linear(in_features=30, out_features=10, bias=True)
    (5): ReLU()
    (6): Linear(in_features=30, out_features=1, bias=True)
  )
)

In [15]:
# Prepare the network for training
epochs = 200
batch_size = 10
# Initialize loss function
loss_fn = torch.nn.MSELoss()
# Initialize optimizer
optm = torch.optim.Adam(params=rmodel.parameters(),lr=1e-2)
# Divide tensors into batch sizes
train_x = X.split(batch_size);
train_y = Y.split(batch_size);

In [ ]:
losses = []
accuracies = []
num_batches = 50
max_accuracy = 0
# best_model = copy.deepcopy(net.state_dict())
for epoch in range(epochs):
    rmodel.train()
    for i in range(num_batches):
        indx = random.randint(0,len(train_x)-1)
        x = train_x[indx]
        y = train_y[indx]
        output = rmodel.forward(x,x.size()[0])
        #print(all(i == net_out[0] for i in net_out))
        loss = torch.sqrt(loss_fn.forward(output,y.reshape(y.size()[0],1)))
        optm.zero_grad()
        loss.backward()
        optm.step()
    with torch.no_grad():
        rmodel.eval()
        net_out = rmodel.forward(test_x,test_x.size()[0])
        loss = torch.sqrt(loss_fn.forward(net_out,test_y.reshape(test_y.size()[0],1)))
        losses.append(loss)
        print(f'EPOCH: {epoch}, Loss: {loss}')